Materiály v tomto notebooku vycházejí z: Companion to Lecture 4 of [Harvard CS109: Data Science](http://cs109.org).

# 1. Pandas: práce s tabulkovými daty

Po části se SymPy se přesuneme k práci s daty.
`pandas` staví na `NumPy` a v prostředí Jupyter notebooků se hodí pro čištění, agregaci a vizualizaci tabulkových dat přes objekt `DataFrame`.

In [ ]:
# pokud nemáme knihovnu pandas, tak ji nainstalujeme
# !pip install pandas -U

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Budeme pracovat s datasetem top filmů z IMDB uloženým v souboru `imdb_top_10000.txt`.

In [ ]:
# prvních několik řádků
!head imdb_top_10000.txt

## 1.1 Sestavení `DataFrame`

Textový soubor je oddělen tabulátory a nemá hlavičku sloupců, proto parametry předáme explicitně do `pd.read_csv`.

In [ ]:
names = ['imdbID', 'title', 'year', 'score', 'votes', 'runtime', 'genres']
data = pd.read_csv('imdb_top_10000.txt', delimiter='\t', names=names)
print("Number of rows: %i" % data.shape[0])
data.head()  # print the first 5 rows

V prostředí Jupyter notebooků se `DataFrame` zobrazuje jako formátovaná HTML tabulka.

## 1.2 Vyčištění `DataFrame`

V datech je potřeba opravit několik věcí:

1. `runtime` je číslo uložené jako text.
2. `genres` obsahuje více žánrů v jednom sloupci.
3. Rok se opakuje i v názvu filmu.

### 1.2.1 Odstranění řádků s chybějícími hodnotami

Použijeme metodu `dropna`.

In [ ]:
data = data.dropna()
print("Number of rows: %i" % data.shape[0])

### 1.2.2 Oprava sloupce `runtime`

Nejprve si ukážeme převod jednoho řetězce typu `'142 mins.'` na číslo.

In [ ]:
dirty = '142 mins.'
number, text = dirty.split(' ')
clean = int(number)
print(number)

Stejný postup pak aplikujeme na celý sloupec pomocí list comprehension.

In [ ]:
clean_runtime = [float(r.split(' ')[0]) for r in data.runtime]
data['runtime'] = clean_runtime
data.head()

### 1.2.3 Rozdělení žánrů do indikátorových sloupců

Sloupec `genres` rozdělíme na binární příznaky (`True`/`False`) pro jednotlivé žánry.

In [ ]:
data.genres[0].split('|')

In [ ]:
# vyrobíme seznam unikátních žánrů

genres = set()
for m in data.genres:
    genres.update(g for g in m.split('|'))
genres = sorted(genres)
print(genres)

In [ ]:
# každý žánr přidáme jako nový sloupec do tabulky
for genre in genres:
    data[genre] = [genre in movie.split('|') for movie in data.genres]

data.head()

### 1.2.4 Odstranění roku z názvu filmu

V názvu je na konci rok v závorce, odstraníme posledních 7 znaků.

In [ ]:
data['title'] = [t[0:-7] for t in data.title]
data.head()

## 1.3 Základní průzkum dat

Začneme souhrnnými statistikami nad hlavními numerickými sloupci.

In [ ]:
data[['score', 'runtime', 'year', 'votes']].describe(include='all')

In [ ]:
# hmmm, a runtime = 0?!
print(len(data[data.runtime == 0]))


In [ ]:
# lepší je nahradit nulu za NaN; runtime = 0 zde nedává smysl
data.loc[data.runtime == 0, 'runtime'] = np.nan

In [ ]:
data.runtime.describe()

### 1.3.1 Základní graf

In [ ]:
plt.hist(data.year, bins=np.arange(1950, 2011+1))
plt.xlabel("Release Year")

## 1.4 Vizualizace pomocí `seaborn`

`seaborn` je nadstavba nad `matplotlib` s pohodlnější syntaxí pro práci s tabulkovými daty.

In [ ]:
# instalace pomocí knihovny seaborn
# !pip install seaborn -U

In [ ]:
import seaborn as sb

In [ ]:
sb.histplot(data, x='year', binwidth=1)
plt.title("Movies per year histogram")
plt.show()

In [ ]:
sb.histplot(data, x='score', bins=20, color='#cccccc', stat='density')
plt.xlabel("IMDB rating")
plt.show()

In [ ]:
sb.histplot(data['runtime'].dropna(), bins=50, color='#cccccc', stat='density')
plt.xlabel("Runtime distribution")
plt.show()

In [ ]:
# horší skóre pro novější filmy?

sb.scatterplot(data=data, x='year', y='score', alpha=0.08, color='k', linewidth=0)
plt.xlabel("Year")
plt.ylabel("IMDB Rating")
plt.show()

In [ ]:
# lepší film -> více lidí hodnotí?

sb.scatterplot(data=data, x='votes', y='score', alpha=0.2, color='k', linewidth=0)
plt.xlabel("Number of Votes")
plt.ylabel("IMDB Rating")
plt.xscale('log')
plt.show()

## 1.5 Výběr řádků podle podmínek

In [ ]:
# nízké skóre ale hodně hlasů
data[(data.votes > 9e4) & (data.score < 5)][['title', 'year', 'score', 'votes', 'genres']]

In [ ]:
# Nejhorší filmy
data[data.score == data.score.min()][['title', 'year', 'score', 'votes', 'genres']]

In [ ]:
# Nejlepší filmy
data[data.score == data.score.max()][['title', 'year', 'score', 'votes', 'genres']]

### 1.5.1 Agregační funkce

Jaké žánry jsou nejčastější?

In [ ]:
# spočítáme počet filmů v každém žánru
genre_counts = data[genres].sum().sort_values(ascending=False)
genre_counts.to_frame(name='genre_count')

Kolik žánrů má film v průměru?

In [ ]:
genre_per_movie = data[genres].sum(axis=1)
print(f"Average movie has {genre_per_movie.mean()} genres")
genre_per_movie.describe()

## 1.6 Vlastnosti skupin (`groupby`)

Filmy rozdělíme podle desetiletí.

In [ ]:
decade = (data.year // 10) * 10

films_with_decade = data[['title', 'year']].copy()
films_with_decade['decade'] = decade

films_with_decade.head()

Objekt [GroupBy](https://pandas.pydata.org/docs/reference/groupby.html) sdružuje řádky se stejnou hodnotou klíče, zde stejné desetiletí.

In [ ]:
# průměrné skore za každou dekádu
decade_mean = data.groupby(decade).score.mean()
decade_mean.name = 'Decade Mean'
print(decade_mean)

In [ ]:
# vykreslíme si graf (předchozí scatter plot) s průměrem hodnocení za dekádu
# Line plot pro průměrné hodnocení za dekádu
plt.plot(decade_mean.index, decade_mean.values, 'o-', color='r', lw=3, label='Decade Average')

# Scatter plot jako překrytí
sb.scatterplot(data=data, x='year', y='score', alpha=0.04, color='k', linewidth=0)

plt.xlabel("Year")
plt.ylabel("Score")
plt.legend(frameon=False)
plt.show()

Můžeme jít dál a spočítat i směrodatnou odchylku v každé dekádě.

In [ ]:
grouped_scores = data.groupby(decade).score

mean = grouped_scores.mean()
std = grouped_scores.std()
print(std)

In [ ]:
# a přikreslíme tento údaj jako "fill" kolem průměru
plt.plot(decade_mean.index, decade_mean.values, 'o-',
        color='r', lw=3, label='Decade Average')
plt.fill_between(decade_mean.index, (decade_mean + std).values,
                 (decade_mean - std).values, color='r', alpha=.2)
sb.scatterplot(data=data, x='year', y='score', alpha=0.04, color='k', linewidth=0)
plt.xlabel("Year")
plt.ylabel("Score")
plt.legend(frameon=False)

Objekt `GroupBy` lze iterovat. Každá iterace vrací dvojici `(klíč, podtabulka)`.
Níže pro každý rok vypíšeme film(y) s nejvyšším hodnocením.

In [ ]:
# nejlepší filmy pro každý rok

for year, subset in data.groupby('year'):
    print(year, subset[subset.score == subset.score.max()].title.values)

## 1.7 Srovnání distribucí podle žánru

Na závěr porovnáme podle žánrů distribuci roku vydání, délky a hodnocení.
Šedě je vždy vykreslená distribuce všech filmů pro referenci.

In [ ]:
# vytvoříme mřížku 4x6 grafů
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8),
                         tight_layout=True)

bins = np.arange(1950, 2013, 3)
for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].year, density=True,
            bins=bins, histtype='stepfilled', color='r', alpha=.3, ec='none')
    ax.hist(data.year, bins=bins, histtype='stepfilled', ec='None', density=True, zorder=0, color='#cccccc')

    ax.annotate(genre, xy=(1955, 3e-2), fontsize=14)
    ax.xaxis.set_ticks(np.arange(1950, 2013, 30))
    ax.set_yticks([])
    ax.set_xlabel('Year')

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8), tight_layout=True)

bins = np.arange(30, 240, 10)

for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].runtime, density=True,
            bins=bins, histtype='stepfilled', color='r', ec='none', alpha=.3)

    ax.hist(data.runtime, bins=bins, density=True,
            histtype='stepfilled', ec='none', color='#cccccc',
            zorder=0)

    ax.set_xticks(np.arange(30, 240, 60))
    ax.set_yticks([])
    ax.set_xlabel("Runtime [min]")
    ax.annotate(genre, xy=(230, .02), ha='right', fontsize=12)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(12, 8), tight_layout=True)

bins = np.arange(0, 10, .5)

for ax, genre in zip(axes.ravel(), genres):
    ax.hist(data[data[genre] == 1].score, density=True,
            bins=bins, histtype='stepfilled', color='r', ec='none', alpha=.3)

    ax.hist(data.score, bins=bins,
            histtype='stepfilled', ec='none', color='#cccccc', density=True,
            zorder=0)

    ax.set_yticks([])
    ax.set_xlabel("Score")
    ax.annotate(genre, xy=(0, .2), ha='left', fontsize=12)